# Train model Stochastic Gradient Descent

## Prepare data

In [18]:
import pandas as pd
import pickle
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

INPUT_DATA_FOLDER = "data/source"
ALL_MERGED_DATA = "all_merged_data.csv"

MODEL_FOLDER = "model"
MODEL_FILE_NAME = "stochastic_gradient_descent_model.pkl"

FEATURES_TO_INCLUDE = [
    'city_resolvedAddress',
    'day_temp',
    'day_humidity',
    'hour_windspeed',
    'hour_conditions',
    'city',
    #'event_start_hour',
    # 'num_regions',
    'hours_last_day',
   # 'vectors'
]
TARGET_FEATURE = 'is_alarm'

# Load dataset from a CSV file
df = pd.read_csv(f"{INPUT_DATA_FOLDER}/{ALL_MERGED_DATA}", sep=";")

# Separate the target variable from the input features
X = df[FEATURES_TO_INCLUDE]
y = df[TARGET_FEATURE]

# Convert all columns to float
X = X.apply(pd.to_numeric, errors='coerce')
y = y.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with default
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)

# Split the dataset into training and testing sets
tss = TimeSeriesSplit(2)
train_index, test_index = list(tss.split(X))[-1]
X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]


## Train model

In [19]:
# Runs for quite a time
# Results in best alpha 0.001
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import GridSearchCV

# sgd = SGDClassifier()

# pipe = Pipeline(steps=[('sgd', sgd)])

# alphas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
# parameters = dict(sgd__alpha=alphas)

# clf = GridSearchCV(pipe, parameters, verbose=10)

# clf.fit(X, y)
# print('Best alpha:', clf.best_estimator_.get_params()['sgd__alpha'])
# print(); print(clf.best_estimator_.get_params()['sgd'])

In [20]:
# Train an SGD model
model = SGDClassifier(loss='squared_error', penalty='l2', alpha=0.001, max_iter=1000)
model.fit(X_train, y_train)
# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

[[ 0.          0.00315885 -0.003349    0.00455018  0.          0.
   0.07221146]]
Accuracy: 0.8308813871010114


In [21]:
print(model.coef_)

[[ 0.          0.00315885 -0.003349    0.00455018  0.          0.
   0.07221146]]


## Save model

In [22]:
with open(f"{MODEL_FOLDER}/{MODEL_FILE_NAME}", 'wb') as f:
    pickle.dump(model, f)

## Calculate confusion matrix

In [23]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[49511   996]
 [ 9304  1093]]
